In [4106]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table
import dash_daq as daq

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

import flask
from flask_cors import CORS

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

import os
from os import environ
import requests
import io

import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt

import socket

import seaborn as sns
from itertools import groupby

In [4107]:
# files_lg=[x for x in os.listdir() if '_lg_' in x and 'business' not in x]
# cols_lg=['Origin','Destination','Carrier','Date','Time','drop','Yield','BidPrice','Customer','Direction','Interval']
# Data_lg=DataFrame(columns=cols_lg)

# for f in files_lg:

#     Data_intermediate=pd.read_csv(f,header=None)

#     Data_intermediate.columns=['Origin','Destination','Carrier','Date','Time','drop','Yield','BidPrice']

#     Data_intermediate['Customer']=f.split('_')[4]
#     Data_intermediate['Direction']=f.split('_')[5]
#     Data_intermediate['Interval']=f.split('_')[6]
#     Data_intermediate=Data_intermediate[cols_lg]

#     Data_lg=pd.concat([Data_lg,Data_intermediate])
#     Data_lg=Data_lg[cols_lg]
    
# Data_lg=Data_lg[[x for x in Data_lg.columns if x!='drop']]
# Data_lg.index=range(Data_lg.shape[0])

# ############################################

# Data_bid=Data_lg[[x for x in cols_lg if x!='Yield' and x!='drop']].rename(columns={'BidPrice': 'Yield'})
# Data_bid['Carrier']='BID'
# Data_lg=Data_lg[[x for x in cols_lg if x!='BidPrice' and x!='drop']]

# ############################################

# files_comp=[x for x in os.listdir() if '_comp_' in x and 'business' not in x]
# cols_comp=['Origin','Destination','Carrier','Date','Time','drop','Yield','Customer','Direction','Interval']
# Data_comp=DataFrame(columns=cols_comp)

# for f in files_comp:

#     Data_intermediate=pd.read_csv(f,header=None)

#     Data_intermediate.columns=['Origin','Destination','Carrier','Date','Time','drop','Yield']

#     Data_intermediate['Customer']=f.split('_')[4]
#     Data_intermediate['Direction']=f.split('_')[5]
#     Data_intermediate['Interval']=f.split('_')[6]
#     Data_intermediate=Data_intermediate[cols_comp]

#     Data_comp=pd.concat([Data_comp,Data_intermediate])
#     Data_comp=Data_comp[cols_comp]
    
# Data_comp=Data_comp[[x for x in Data_comp.columns if x!='drop']]
# Data_comp.index=range(Data_comp.shape[0])

# ############################################

# Data=pd.concat([Data_lg,Data_comp,Data_bid])

# Data['Date']=pd.to_datetime(Data['Date'])

# Data['Interval']=Data['Interval'].astype('int')

# Dash

In [4108]:
Data=pd.read_csv('Data_forhome.csv',sep=';')

Data['Direction']=Data['Direction'].apply(lambda x: 1 if x=='outbound' else 0)

marks_interval=dict(zip([1,2,4,5,6,7,8,10,12],
    [str(x)+'m' if x<=60 else str(int(round(x/60)))+'h' for x in sorted(Data['Interval'].unique())]))
interval_map=dict(zip(sorted(Data['Interval'].unique()),
[1,2,4,5,6,7,8,10,12]))
Data['Interval']=Data['Interval'].apply(lambda x: interval_map[x])

Data['Date']=pd.to_datetime(Data['Date'],format='%d.%m.%Y')
Data['Weekday']=Data['Date'].apply(lambda x: x.weekday())

Data['Yield']=Data['Yield'].apply(lambda x: None if 'NULL' in str(x) else float(x))

In [4109]:
#Data=Data.dropna(how='any')
#Data['Yield']=Data['Yield'].apply(lambda x: None if pd.isnull(x) or x==np.nan else x)

In [4110]:
Data=Data.sort_values(by=['Origin','Destination','Carrier','Date','Time','Customer','Direction','Interval'])

In [4111]:
app = dash.Dash(__name__)

In [4112]:
intermediate=Data[['Origin','Destination','Time','Customer','Direction','Interval','Weekday']].drop_duplicates()
checkbox_values=intermediate\
.sort_values(by=['Origin','Destination','Time','Customer','Direction','Interval','Weekday']).transpose().values.tolist()

In [4113]:
mapbox_access_token = 'pk.eyJ1IjoiamFja2x1byIsImEiOiJjajNlcnh3MzEwMHZtMzNueGw3NWw5ZXF5In0.fk8k06T96Ml9CLGgKmk81w'

In [4114]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [4115]:
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return int((dt - epoch).total_seconds())#* 1000.0

def get_marks_from_start_end(start, end):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    result = []
    current = start
    while current <= end:
        result.append(current)
        current += relativedelta(hours=1)
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in result}

def get_marks_from_start_end_2(seq):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in seq}

In [4116]:
# all_marks_out=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
# all_marks_in=get_marks_from_start_end(min(checkbox_values[3]),max(checkbox_values[3]))

# marks_lg=get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates())
# all_marks=dict(zip([x for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000],
# [all_marks[x] for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000]))
# all_marks.update(marks_lg)

#all_marks=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))

In [4117]:
marks=dict(zip([np.sqrt(x) for x in sorted(intermediate['Interval'].unique())],
    [str(x)+'m' if x<=60 else str(int(round(x/60)))+'h' for x in sorted(intermediate['Interval'].unique())])),

In [4118]:
colors_carrier=dict(zip(['LG','TP','FR','U2','AZ','OS','KL','SK','V7','BID'],
                        ['blue','#71BF44','#f1c933','#FF7D1C','#2c692c','#E71C23','#00a1e4','#2B0F73','#EB2226','white']))

In [4119]:
PAGE_SIZE = 20

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        html.Div([html.Div(style={'width': '10%', 'display': 'inline-block'}),
        html.Img(src=app.get_asset_url('LuxairGroup.jpg'), 
              style = {'backgroundColor' : 'black','display': 'inline-block', 'vertical-align': 'middle', 
                       'width': '10%', 'height': '7em'}),
        html.H4(children='Price Monitoring System',style={
            'textAlign': 'center',
            'color': 'white','height': '1em', 'display': 'inline-block', 'width': '64%'}),
            html.Img(src=app.get_asset_url('Luxair.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle',
                       'width': '10%', 'height': '7em'}),
        html.Div(style={'width': '10%', 'display': 'inline-block', 'vertical-align': 'middle'})
                 ],
        style={'margin-bottom': '0em', 'margin-top': '-2em'})
     ,
    # ROUTE DROPDOWN AND DATE SLIDER
    
        # DROPDOWN
    html.Div([html.Div(style={'width': '10%', 'margin-bottom': '1em', 'display': 'inline-block'}),
    
    html.Div([html.Div([html.Div([html.Label('Origin', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_origin',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[0]))],
    placeholder="Select origin", 
    value=None,
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '2em', 'display': 'inline-block'})
    ,
    html.Div([html.Label('Destination', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_destination',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[1]))],
    placeholder="Select destination",
    value=None,
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '1em', 'display': 'inline-block'}),
    html.Div([html.Label('Time', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_time',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[2]))],
    placeholder="Select time",
    value=None,
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '1em', 'display': 'inline-block'})],          
    style={'margin-bottom': '0.5em', 'width': '90%', 'margin-left': '1em'}),
    html.Div([html.Div([html.Label('Customer Segment', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.RadioItems(id = 'radio_customer',
    labelStyle={'display': 'inline-block'},
    value=None,
    className='char-btn')],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '5em', 'display': 'inline-block'})
    ,
    html.Div([html.Label('Direction', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    daq.Slider(id = 'slider_direction',
    labelPosition='center',
    min=0,
    max=1,
    marks=dict(zip([0,1],['inbound','outbound'])),
    step=None,
    included=False,
    size=30,
    handleLabel='',
    vertical=True,
    color='red',
    value=1)],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '3em', 'display': 'inline-block'})],          
    style={'margin-bottom': '0.5em', 'width': '90%', 'margin-left': '1em'})
    ,
    html.Div([html.Label('Interval', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Slider(id='slider_interval',
    min=0,
    max=12,
    marks=marks_interval,
    step=None,
    included=False,
    value=1
    )],
    style={'margin_top': '0em', 'margin-bottom': '2em', 'width': '60%', 'margin-left': '5em'})],
    style={'width': '45%', 'margin-bottom': '1em', 'margin-top': '0','display': 'inline-block', 'background-color': 'black'}),
    
    html.Div([html.Div(style={'width': '100%', 'margin-bottom': '-2em', 'margin-top': '2em'}),
    #dcc.Checklist(options={'label': 'Apply time filter', 'value': 'filter'}),
    html.Div([html.Label('Weekday', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_weekday',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[6]))],
    placeholder="Select weekday", 
    value=None,
    multi=True)],
    style={'width': '30%', 'margin-bottom': '0.5em', 'margin-left': '12em'}),
    html.Div([html.Label('Departure Date', style={'textAlign': 'center', 'font-weight': 'bold'}),
    dcc.DatePickerRange(
        id='picker_date',
        min_date_allowed=Data['Date'].min(),
        max_date_allowed=Data['Date'].max(),
        initial_visible_month=datetime.datetime.today(),
        start_date=None,
        end_date=None
    ), html.Button("Clear Date",id='button_reset',n_clicks=0)],style={'width': '80%','margin-left': '4em', 'margin-bottom': '1.5em'})],
    style={'width': '40%', 'margin-bottom': '1.5em', 'margin-top': '1em', 'margin-left': '3em','display': 'inline-block',
          'background-color': 'black'}),          
              
    html.Div(style={'width': '5%', 'margin-bottom': '0.5em', 'margin-top': '-3em', 'display': 'inline-block'})
             ],
    style={'width': '90%', 'margin-bottom': '0.5em', 'margin-top': '-2em'}),
    # GRAPH CONTAINER                   
    html.Div([dcc.Graph(
                    id='graph')],
    style={'width': '90%', 'float': 'center', 'margin-top': '0.5em', 'margin-left': '4em'})
        ],
        style={'background-color': '#211A23'})

In [4120]:
@app.callback(
    Output("dropdown_origin", "options"),
    [Input("dropdown_destination", "value"),
    Input("dropdown_time", "value"), 
    Input("radio_customer", "value"),
    Input("slider_direction", "value"),
    Input("slider_interval", "value")]
    )
def set_dropdown_origin(values_destination,values_time,values_customer,values_direction,values_interval):
    
    if values_destination is None:
        values_destination=set(checkbox_values[1])
    if values_time is None:
        values_time=set(checkbox_values[2])
    if values_customer is None:
        values_customer=set(checkbox_values[3]) 
    
    subset=set([checkbox_values[0][i] for i in range(len(checkbox_values[0]))\
        if checkbox_values[1][i] in values_destination\
        and checkbox_values[2][i] in values_time\
        and checkbox_values[3][i] in values_customer\
        and checkbox_values[4][i]==values_direction\
        and checkbox_values[5][i]==values_interval])
        
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [4121]:
@app.callback(
    Output("dropdown_destination", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_time", "value"), 
    Input("radio_customer", "value"),
    Input("slider_direction", "value"),
    Input("slider_interval", "value")]
    )
def set_dropdown_destination(values_origin,values_time,values_customer,values_direction,values_interval):

    if values_origin is None:
        values_origin=set(checkbox_values[0])
    if values_time is None:
        values_time=set(checkbox_values[2])
    if values_customer is None:
        values_customer=set(checkbox_values[3])      
    
    subset=set([checkbox_values[1][i] for i in range(len(checkbox_values[1]))\
        if checkbox_values[0][i] in values_origin\
        and checkbox_values[2][i] in values_time\
        and checkbox_values[3][i] in values_customer\
        and checkbox_values[4][i]==values_direction\
        and checkbox_values[5][i]==values_interval])
        
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values   
    
##########################################################

In [4122]:
@app.callback(
    Output("dropdown_time", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("radio_customer", "value"),
    Input("slider_direction", "value"),
    Input("slider_interval", "value")]
    )
def set_dropdown_time(values_origin,values_destination,values_customer,values_direction,values_interval):

    if values_origin is None:
        values_origin=set(checkbox_values[0])
    if values_destination is None:
        values_destination=set(checkbox_values[1])
    if values_customer is None:
        values_customer=set(checkbox_values[3])      
    
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
        if checkbox_values[0][i] in values_origin\
        and checkbox_values[1][i] in values_destination\
        and checkbox_values[3][i] in values_customer\
        and checkbox_values[4][i]==values_direction\
        and checkbox_values[5][i]==values_interval])
        
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values   
    
##########################################################

In [4123]:
@app.callback(
    Output("radio_customer", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("dropdown_time", "value"),
    Input("slider_direction", "value"),
    Input("slider_interval", "value")]
    )
def set_dropdown_customer(values_origin,values_destination,values_time,values_direction,values_interval):

    if values_origin is None:
        values_origin=set(checkbox_values[0])
    if values_destination is None:
        values_destination=set(checkbox_values[1])
    if values_time is None:
        values_time=set(checkbox_values[2])    
    
    subset=set([checkbox_values[3][i] for i in range(len(checkbox_values[3]))\
        if checkbox_values[0][i] in values_origin\
        and checkbox_values[1][i] in values_destination\
        and checkbox_values[2][i] in values_time\
        and checkbox_values[4][i]==values_direction\
        and checkbox_values[5][i]==values_interval])
        
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values   
    
##########################################################

In [4124]:
@app.callback(
    [Output("slider_direction", "min"),
    Output("slider_direction", "max"),
    Output("slider_direction", "marks")],
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("dropdown_time", "value"),
    Input("radio_customer", "value"),
    Input("slider_interval", "value")]
    )
def set_dropdown_direction(values_origin,values_destination,values_time,values_customer,values_interval):

    if values_origin is None:
        values_origin=set(checkbox_values[0])
    if values_destination is None:
        values_destination=set(checkbox_values[1])
    if values_time is None:
        values_time=set(checkbox_values[2])
    if values_customer is None:
        values_customer=set(checkbox_values[3])   
    
    subset=set([checkbox_values[4][i] for i in range(len(checkbox_values[4]))\
        if checkbox_values[0][i] in values_origin\
        and checkbox_values[1][i] in values_destination\
        and checkbox_values[2][i] in values_time\
        and checkbox_values[3][i] in values_customer\
        and checkbox_values[5][i]==values_interval])
    minimum=0
    maximum=1
    marks_all=dict(zip([0,1],['inbound','outbound']))
    marks=dict(zip([i for i in subset],
    [marks_all[i] for i in subset]))
    
    return minimum, maximum, marks   
    
##########################################################

In [4125]:
@app.callback(
    [Output("slider_interval", "min"),
    Output("slider_interval", "max"),
    Output("slider_interval", "marks")],
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("dropdown_time", "value"),
    Input("radio_customer", "value"),
    Input("slider_direction",'value')]
    )
def set_dropdown_direction(values_origin,values_destination,values_time,values_customer,values_direction):

    if values_origin is None:
        values_origin=set(checkbox_values[0])
    if values_destination is None:
        values_destination=set(checkbox_values[1])
    if values_time is None:
        values_time=set(checkbox_values[2])
    if values_customer is None:
        values_customer=set(checkbox_values[3])    
    
    subset=set([checkbox_values[5][i] for i in range(len(checkbox_values[5]))\
        if checkbox_values[0][i] in values_origin\
        and checkbox_values[1][i] in values_destination\
        and checkbox_values[2][i] in values_time\
        and checkbox_values[3][i] in values_customer\
        and checkbox_values[4][i]==values_direction])
    minimum=1
    maximum=12
    marks=dict(zip([i for i in subset],
    [marks_interval[i] for i in subset]))
    
    return minimum, maximum, marks   
    
##########################################################

In [4126]:
@app.callback(
    Output("dropdown_weekday", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("dropdown_time", "value"),
    Input("radio_customer", "value"), 
    Input("slider_direction", "value"),
    Input("slider_interval", "value")]
    )
def set_dropdown_weekday(values_origin,values_destination,values_time,values_customer,values_direction,values_interval):

    if values_origin is None:
        values_origin=set(checkbox_values[0])
    if values_destination is None:
        values_destination=set(checkbox_values[1])
    if values_time is None:
        values_time=set(checkbox_values[2])    
    if values_customer is None:
        values_customer=set(checkbox_values[3]) 
        
    subset=set([checkbox_values[6][i] for i in range(len(checkbox_values[6]))\
        if checkbox_values[0][i] in values_origin\
        and checkbox_values[1][i] in values_destination\
        and checkbox_values[2][i] in values_time\
        and checkbox_values[3][i] in values_customer\
        and checkbox_values[4][i]==values_direction\
        and checkbox_values[5][i]==values_interval])
        
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values   
    
##########################################################

In [4127]:
# @app.callback(
#     [Output('picker_date', 'start_date'),
#     Output('picker_date', 'end_date')],
#     [
#         Input('button', 'n_clicks'),
#     ],
#     [
#         State('picker_date', 'start_date'),
#     ]
# )
# def clear_date(n_clicks, current_selected_date):
#     ''' clears the date when button is clicked'''

#     if (n_clicks is not None) and (n_clicks > 0):
#         # =============================== neither of both following lines work 
#         # return ''
#         return None
#     else:
#         return current_selected_date


# if __name__ == '__main__':
#     app.run_server(debug=True)

In [4128]:
# @app.callback(
#     Output('radio_customer', 'value'),
#     [Input('radio_customer', 'options')])
# def set_customer_value(available_options_customer):
#     return None

In [4129]:
@app.callback(Output('graph', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("dropdown_time", "value"),
    Input("radio_customer", "value"), 
    Input("slider_direction", "value"),
    Input("slider_interval", "value"),
    Input("dropdown_weekday",'value'),
    Input("picker_date","start_date"),
    Input("picker_date","end_date")]
    )
def update_plot(values_origin,values_destination,values_time,values_customer,values_direction,values_interval,
               values_weekday,values_startdate,values_enddate):
    
    if values_weekday is None:
        values_weekday = []
    
    dff_plot = Data.loc[(Data['Origin']==values_origin)\
                   &(Data['Destination']==values_destination)\
                   &(Data['Time']==values_time)\
                   &(Data['Customer']==values_customer)\
                   &(Data['Direction']==values_direction)\
                   &(Data['Interval']==values_interval),:]
    
    dff2_plot=DataFrame(columns=dff_plot.columns)
    
    if (len(values_weekday)>0)&(values_enddate is None):
        dff2_plot=dff_plot.loc[dff_plot['Weekday'].apply(lambda x: x in values_weekday),:]
    elif (len(values_weekday)==0)&(values_enddate is not None):
        dff2_plot=dff_plot.loc[(dff_plot['Date']>=values_startdate)&(dff_plot['Date']<=values_enddate),:]
    elif (len(values_weekday)>0)&(values_enddate is not None):
        dff2_plot=dff_plot.loc[dff_plot['Weekday'].apply(lambda x: x in values_weekday)\
                    &(dff_plot['Date']>=values_startdate)&(dff_plot['Date']<=values_enddate),:]
        
    carriers=dff_plot['Carrier'].unique()
    
    if dff2_plot.shape[0]==0:
    
        traces = []

        for j in carriers:
             
            x=pd.date_range(dff_plot['Date'].min(),dff_plot['Date'].max())
            y=[None]*len(x)
            yield_dict=dict(zip(x,y))
            x_update=dff_plot.loc[dff_plot['Carrier']==j,'Date'].tolist()
            y_update=dff_plot.loc[dff_plot['Carrier']==j,'Yield'].tolist()
            yield_dict.update(dict(zip(x_update,y_update)))
            x=[i for i in yield_dict.keys()]
            y=[i for i in yield_dict.values()]

            traces.append(go.Scatter(
            x=x,
            y=y,
            mode = 'lines+markers',
            name = j,
            #+'/'+dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_time_comp'].unique().tolist()[0],
            connectgaps=False,
            line = dict(
            dash = 'solid',
            color =colors_carrier[j],
            width = 2
                     )
                ))     
   
    else:
        
        traces=[]
        
        for j in carriers:
        
            x=pd.date_range(dff_plot['Date'].min(),dff_plot['Date'].max())
            y=[None]*len(x)
            yield_dict=dict(zip(x,y))
            x_update=dff_plot.loc[dff_plot['Carrier']==j,'Date'].tolist()
            y_update=dff_plot.loc[dff_plot['Carrier']==j,'Yield'].tolist()
            yield_dict.update(dict(zip(x_update,y_update)))
            x=[i for i in yield_dict.keys()]
            y=[i for i in yield_dict.values()]

            traces.append(go.Scatter(
            x=x,
            y=y,
            mode = 'lines+markers',
            name=j,
            showlegend=False,
            #+'/'+dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_time_comp'].unique().tolist()[0],
            connectgaps=False,
            opacity=0.1,
            hoverinfo='none',
            line = dict(
            dash = 'solid',
            color =colors_carrier[j],
            width = 2
                     )
                )) 

        for j in carriers:
        
            x=pd.date_range(dff2_plot['Date'].min(),dff2_plot['Date'].max())
            y=[None]*len(x)
            yield_dict=dict(zip(x,y))
            x_update=dff2_plot.loc[dff2_plot['Carrier']==j,'Date'].tolist()
            y_update=dff2_plot.loc[dff2_plot['Carrier']==j,'Yield'].tolist()
            yield_dict.update(dict(zip(x_update,y_update)))
            x=[i for i in yield_dict.keys()]
            y=[i for i in yield_dict.values()]

            traces.append(go.Scatter(
            x=x,
            y=y,
            mode = 'lines+markers',
            name = j,
            #+'/'+dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_time_comp'].unique().tolist()[0],
            connectgaps=False,
            line = dict(
            dash = 'solid',
            color =colors_carrier[j],
            width = 2
                     )
                ))
                   
    return {'data': traces, 'layout': 
    go.Layout(title=go.layout.Title(
    text='Outbound Flight',
    xref='paper',
    x=0.5),
    xaxis={'title': 'Departure Date'},
    yaxis={'title': 'Yield', 'range': [0,max([dff_plot['Yield'].max()])]},
    height=450,
    plot_bgcolor='#000000',
    paper_bgcolor='#000000')}         

In [4130]:
@app.callback([Output('picker_date','start_date'),
              Output('picker_date','end_date')],
             [Input('button_reset','n_clicks')])
def update(reset):
    return None,None

In [4131]:
layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(
        l=30,
        r=30,
        b=20,
        t=40
    ),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation='h'),
    title='Satellite Overview',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style="light",
        center=dict(
            lon=-78.05,
            lat=42.54
        ),
        zoom=7,
    )
)

In [4132]:
if __name__ == '__main__':
    #app.run_server(debug=False,host='10.0.0.3',port=4000)
    app.run_server(debug=False,host='localhost',port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jul/2019 22:38:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:46] "GET /assets/style.css?m=1562531910.8164842 HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2019 22:38:47] "POST /_dash-up